In [ ]:
import datetime
import pandas as pd
import os
import sys
import glob
# Importación de librerías para manejo de Excel
import openpyxl   # Para trabajar con archivos Excel (.xlsx)

print("Librerías cargadas correctamente.")

In [ ]:
R_Recaudo =r"C:\Users\osmarrincon\Downloads\RECAUDO CORTE A NOV-DIC-FEBRERO 2025  ok.xlsx"
RS_Recaudo =r"C:\Users\osmarrincon\Downloads\RECAUDO CORTE A NOV-DIC-FEBRERO 2025.xlsx"
R_Pila_SIE_I = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Compensación\_Pila_SIE\Pila I"
R_Pila_SIE_IP = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Compensación\_Pila_SIE\Pila IP"

In [ ]:
Df_Recaudo = pd.read_excel(R_Recaudo, sheet_name="RELACION RECAUDO", header=0, dtype=str)

# Buscar todos los archivos .TXT en la carpeta R_Pila_SIE_I
txt_files = glob.glob(os.path.join(R_Pila_SIE_I, "*.TXT"))
# Leer cada archivo y almacenarlo en una lista de DataFrames
dfs = [pd.read_csv(file, delimiter="|", encoding="ANSI", dtype=str) for file in txt_files]
# Concatenar todos los DataFrames en uno solo
Df_Pila_I = pd.concat(dfs, ignore_index=True)

# Buscar todos los archivos .TXT en la carpeta R_Pila_SIE_IP
txt_files = glob.glob(os.path.join(R_Pila_SIE_IP, "*.TXT"))
# Leer cada archivo y almacenarlo en una lista de DataFrames
dfs = [pd.read_csv(file, delimiter="|", encoding="ANSI", header=None, dtype=str) for file in txt_files]
# Concatenar todos los DataFrames en uno solo
Df_Pila_IP = pd.concat(dfs, ignore_index=True)
Df_Pila_IP = Df_Pila_IP.applymap(lambda x: 1 if x == "X" else (0 if pd.isna(x) or x == "" else x))

# Asignar los encabezados de Df_Pila_I a Df_Pila_IP para que tengan las mismas columnas
Df_Pila_IP.columns = Df_Pila_I.columns
# Concatenar Df_Pila_I y Df_Pila_IP, colocando los registros de Df_Pila_IP debajo de los de Df_Pila_I
Df_Pila_SIE = pd.concat([Df_Pila_I, Df_Pila_IP], ignore_index=True)

In [ ]:
Df_Pila_SIE = Df_Pila_SIE[Df_Pila_SIE["Correcciones"] != "A"]
cols_keep = [
    "N° Identificación Aportante", "Perido Pago", "Tipo Documento Cotizante",
    "N° Identificación Cotizante", "ING", "RET", "Días Cotizados",
    "Cotización Obligatoria", "Número Planilla", "Fecha Planilla"
]
Df_Pila_SIE = Df_Pila_SIE[cols_keep]

In [ ]:
from datetime import datetime
import pandas as pd

# Obtener el valor mínimo de la columna "periodo_pago"
min_period = min(Df_Recaudo["periodo_pago"].unique())
# Convertir el valor mínimo a objeto datetime (formato "yyyy-mm")
min_date = datetime.strptime(min_period, "%Y-%m")

# Obtener el primer día del mes actual
current_date = datetime.today().replace(day=1)

# Generar un rango de fechas mensuales desde min_date hasta current_date
date_range = pd.date_range(start=min_date, end=current_date, freq='MS')

# Convertir cada fecha del rango a formato "yyyy-mm"
unique_periods = [d.strftime("%Y-%m") for d in date_range]

# Para cada valor en el rango, crear una columna vacía en Df_Recaudo
for period in unique_periods:
    Df_Recaudo[period] = None

In [ ]:
# Renombrar columnas en Df_Pila_SIE para que se ajusten a los nombres de Df_Recaudo
Df_Pila_SIE = Df_Pila_SIE.rename(columns={
    "N° Identificación Aportante": "numero_identificacion_aportante",
    "Perido Pago": "periodo_pago",
    "Tipo Documento Cotizante": "tipo_identificacion_cotizante",
    "N° Identificación Cotizante": "numero_identificacion_cotizante"
})

# Crear el diccionario mapping con las columnas renombradas
mapping = Df_Pila_SIE.set_index(
    ["numero_identificacion_aportante", "tipo_identificacion_cotizante", "numero_identificacion_cotizante", "periodo_pago"]
)["Cotización Obligatoria"].to_dict()

def assign_cotizacion_all(row):
    key_prefix = (
        row["numero_identificacion_aportante"],
        row["tipo_identificacion_cotizante"],
        row["numero_identificacion_cotizante"]
    )
    for key, cotizacion in mapping.items():
        if key[:3] == key_prefix:
            period = key[3]
            # Si la columna para ese período existe en Df_Recaudo, se asigna el valor
            if period in row.index:
                row[period] = cotizacion
    return row

Df_Recaudo = Df_Recaudo.apply(assign_cotizacion_all, axis=1)

In [ ]:
Df_Recaudo.to_excel(RS_Recaudo, index=False)